Import the rsa module we wrote

In [1]:
import rsa

Generate a public and private key

In [ ]:
# Generate two distinct random prime integers (normally would be random but you can choose them here)
p = 1000000007
q = 2412134003

private_key, public_key = rsa.generate_key(p,q)
print("Public Key:", public_key)
print("Private Key:", private_key)

Public Key: (2412134019884938021, 1037901271948760861)
Private Key: (2412134019884938021, 84385038535630421)


Enter a message

In [ ]:
# Enter a message to encrypt
MESSAGE = "ABC"

Convert the message to an integer

In [25]:
def convert_message_to_int(message):
    """ Convert a message written in letters to integers using python's ord function.
    
    Args:
        message: str containing your message

    Returns:
        message_as_int: the message converted to an integer (int)
    """
    message_as_int = ""
    for i in message:
        int_letter = "{:03d}".format(ord(i))  # convert single letter to integer
        message_as_int += str(int_letter)
    message_as_int = int(message_as_int)
    return message_as_int

message_as_int = convert_message_to_int(MESSAGE)

print("Your message as an integer:", message_as_int)

# The message must be smaller than p*q to be encrypted correctly
if message_as_int >= p*q:
    raise ValueError("The message in integer form must be smaller than p*q. Choose a shorter message or larger prime numbers.")

Your message as an integer: 104101108108111


Encrypt the message

In [29]:
encrypted_message = rsa.encrypt(public_key, message_as_int)
encrypted_message = str(encrypted_message) # needs to be a string to send over email, but we'll convert it back afterwards
print(encrypted_message)

803191198881463902


Send the message to your recipient over email

In [ ]:
import smtplib
from email.mime.text import MIMEText
from dotenv import load_dotenv
import os

# Load variables from the .env file
load_dotenv()

# Access the app password, sender email, and receiver email
app_pass = os.getenv("APP_PASS")  # Retrieve the API key
my_email = os.getenv("MY_EMAIL")  # Retrieve sender email
receiver_email = os.getenv("RECEIVER_EMAIL") # Retrieve receiver email
if not app_pass:
    raise EnvironmentError("APP_PASS not found in .env file or environment variables")
if not my_email:
    raise EnvironmentError("MY_EMAIL not found in .env file or environment variables")
if not receiver_email:
    raise EnvironmentError("RECEIVER_EMAIL not found in .env file or environment variables")


def send_email(message, receiver_email):

    subject = "You got an encrypted message!"
    message = message
    sender_email = my_email
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    smtp_username = my_email
    smtp_password = app_pass

    msg = MIMEText(message)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = receiver_email

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(smtp_username, smtp_password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
        # print("Email notification sent successfully!")
    except Exception as e:
        print(f"Failed to send email notification: {e}")


# Send an email to your recipient containing the encrypted message
send_email(encrypted_message, receiver_email)


Read the encrypted message directly from your inbox and display it

In [35]:
import imaplib
import email
from dotenv import load_dotenv
import os

# Access the app password
load_dotenv()
app_pass = os.getenv('APP_PASS')  # Retrieve the API key
my_email = os.getenv('MY_EMAIL')
if not app_pass:
    raise EnvironmentError("APP_PASS not found in .env file or environment variables")
if not my_email:
    raise EnvironmentError("MY_EMAIL not found in .env file or environment variables")


# Gmail IMAP settings
IMAP_SERVER = 'imap.gmail.com'
EMAIL = my_email
PASSWORD = app_pass

def read_emails():
    # Connect to Gmail
    mail = imaplib.IMAP4_SSL(IMAP_SERVER)
    mail.login(EMAIL, PASSWORD)

    # Select the inbox
    mail.select("inbox")

    # Search for all emails
    status, messages = mail.search(None, 'ALL')
    email_ids = messages[0].split()

    for e_id in email_ids[-1:]:  # Fetch the last 1 emails
        status, msg_data = mail.fetch(e_id, '(RFC822)')
        for response_part in msg_data:
            if isinstance(response_part, tuple):
                msg = email.message_from_bytes(response_part[1])

                if msg.is_multipart():
                    for part in msg.walk():
                        if part.get_content_type() == "text/plain":
                            message = part.get_payload(decode=True).decode()
                else:
                    message = msg.get_payload(decode=True).decode()

    mail.logout()

    return message

if __name__ == "__main__":
    received_message = int(read_emails().split()[0])
    print(received_message)


52


Decrypt the message, convert back to letters, and display it

In [32]:
received_message = int(encrypted_message)
print(received_message)

803191198881463902


In [31]:
def convert_int_to_message(message_as_int):
    """ Convert an integer back to a message using python's chr function.

    Args:
        message_as_int: int representing the encoded message

    Returns:
        message: the decoded string message
    """
    message_as_str = str(message_as_int) + "0"  # Convert integer to string for processing
    message = ""
    for i in range(0, len(message_as_str), 3):  # Process in chunks of 3
        int_letter = int(message_as_str[i:i+2])  # Convert chunk back to integer
        message += chr(int_letter)  # Convert integer to character
    return message


# Decrypt the message
decrypted_message = rsa.decrypt(private_key, received_message)

# Convert back to letters
decrypted_message = convert_int_to_message(decrypted_message)

print("Decrypted message:", decrypted_message)

Decrypted message: 



 
